In [1]:
# -*- coding: utf-8 -*-
# https://www.sec.gov/about/forms/form4data.pdf usefull document
# Maybe use this for checking news feelings 
#    https://www.google.com/finance/company_news?q= + tradingSymbolUpper


#import smtplib # only used for Email
import datetime
##from twilio.rest import TwilioRestClient # for sending Texts
import feedparser ## May need this but currently throwing an error
# needed to run CMD  "pip install feedparser"
import openpyxl
import xml.etree.ElementTree as ET
import requests
import bs4
import time
#import io # not used
#import itertools as IT # not used
#import pandas as pd # not used yet - TD
#import numpy as np
import csv
#pip install -U nltk - NEEDED
#python -m nltk.downloader all - NEEDED and takes a long time
#python -m nltk.downloader -d /usr/local/share/nltk_data all - to make it central?
from nltk import sent_tokenize, word_tokenize, pos_tag
# the next two are for TextBlob
#pip install -U textblob
#python -m textblob.download_corpora
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer



In [3]:

#CompanyNameList = []
#TickerList = []
           #ProcessedList = [] # seems like it is not used
           #ReportingOwnerRelationshipList = [] # Not used
TransactionSharesList = [] # List of how many shares for the entery
PricePerShareList = [] #List of the price per share for one entery
TotalValueList = [] #List of the total value of the stock sold or boaugh in a given entery
transactionCodeList = [] 
# list of the Transation Codes aka selling or buying,  We generally only care about P (purchasing) or S (selling)
DorIList = [] # list of the transactions if the where direct or Inderiect (done by the person or not)
#portfolio = []
#bought_price = []
stocks_sent = [] # List of enteries (link based) that have currently been seen this script has been started
#checked = []

def FeedLinkParse(link):
    try: # Try and get the data if it does not time out or anything
        pageText=pageTexts=''
        #here lets find just the important text of each page
        res = requests.get(link,\
            timeout=5,\
            headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.103 Safari/537.36"}, \
            params={'type':'upcoming'})
            # Time out after 5 seconds
            # Adding User-Agent and Params makes it less likely that the request will get blocked
        FeedSoup = bs4.BeautifulSoup(res.text, 'html.parser') #Soup do your beautiful job
        for p in FeedSoup.find_all('p'): # now find all the "p" (aka paragraphs) 
            pageText=p.get_text()
            pageTexts=pageTexts + pageText
        blob=TextBlob(pageTexts)
        #print (pageTexts)
        #print (str(blob.sentiment.polarity) + " " + str(blob.sentiment.subjectivity) )
        return [blob.sentiment.polarity,blob.sentiment.subjectivity]

        
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        #print e # Have the same Return here TED!!!!! C
        # Currently TDYS is a good test case
        return [0.0,0.0]
        pass
    
def GoogleNewsFeed(tradingSymbolUpper):
    FeedSite=FeedSites=[]
    polarities= []
    subjectivities=[]
    url = 'https://www.google.com/finance/company_news?q='+ tradingSymbolUpper + '&output=rss'
    try:
        feed = feedparser.parse(url)
        today = datetime.datetime.today().strftime('%Y/%m/%d %H:%M') 
        #today = today.strftime('%Y/%m/%d %H:%M') 

        for item in feed['entries']: 
            FeedSite=[]
            #For all the results from the news feed get the title, link , date and description
            title=item.title
            link=item.link

            pubDate=item.published
            pubDateTime= datetime.datetime.strptime(pubDate, '%a, %d %b %Y %H:%M:%S %Z') # convert it to a real date time

            description =bs4.BeautifulSoup(item.summary, 'html.parser').get_text() # Make it more like English
            blob=TextBlob(description)

            Results=FeedLinkParse(link) # Go to the links and return the Polarity and Subjectivity

            polarities.append(Results[0])
            subjectivities.append(Results[1]) 
            # Make a new "row"
            FeedSite.append(tradingSymbolUpper)
            FeedSite.append(today)
            FeedSite.append(link)
            FeedSite.append(pubDateTime)
            FeedSite.append(Results[0])
            FeedSite.append(Results[1]) 
            with open('StockPulse.csv', 'ab') as f:
                writer = csv.writer(f)
                writer.writerow(FeedSite)
        if polarities==[]:
            avgPol=None
            avgSub=None
        else:
            avgPol=sum(polarities)/len(polarities)
            avgSub=sum(subjectivities)/len(subjectivities)
        return (avgPol, avgSub)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        #print e # Have the same Return here TED!!!!! C
        # Currently TDYS is a good test case
        pass
#----------------------------------------------------------------------------------#
#SCAN EDGAR AND SCRAPE XML

def scrape_xml(link):
    TotalValue = 0
    transactionCodeList = []
    DorIList = []
    #TitleList = []
    span = [] # used for getting data from Yahoo.com
    tranPPSlist = [] #Transaction Price Per Share
    tranSlist = [] # Entery list of the count of shares
    PostSharesList = [] # howmany shares they had afer the sale or purchase
    BeforeSharesList = [] # howmany shares they had before the sale or purchase
    Entry = [] # Ted added 2016-12-31
    today = datetime.datetime.today() # Used for error reporting
    today = today.strftime('%Y/%m/%d %H:%M') # Used for error reporting

    headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.103 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
    "accept-encoding": "gzip, deflate, sdch",
    "accept-language": "en-US,en;q=0.8",
    }
    res = requests.get(link, headers=headers)
    time.sleep(2)
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    time.sleep(2)
    #try:
    for a in soup.find_all('a'):
        TotalValue =0
        officerTitle = None
        isTenPercentOwner = None
        isDirector = None
        isOfficer = None
        Attorney= None
        if a.getText()[-4:] == '.xml':
            address = 'http://www.sec.gov' + a['href']
            #print ('Scraping XML on ' + str(today) + ' at ' + str(link))
            #print ('Scraping XML '+ str(link)) # keeping it Simple
            res = requests.get(address, headers=headers, timeout=None)
            tree = ET.fromstring(res.content)
            
            if tree.find('reportingOwner/reportingOwnerRelationship/isDirector')== None:
                isDirector = None
            elif tree.find('reportingOwner/reportingOwnerRelationship/isDirector').text == '1':
                isDirector = True
            elif tree.find('reportingOwner/reportingOwnerRelationship/isDirector').text == '0':
                isDirector = None
            elif tree.find('reportingOwner/reportingOwnerRelationship/isDirector').text == 'false':
                isDirector = None
            else:
                isDirector = tree.find('reportingOwner/reportingOwnerRelationship/isDirector')
                isDirector = isDirector.text
                
            if tree.find('reportingOwner/reportingOwnerRelationship/isOfficer')== None:
                isOfficer = None
            elif tree.find('reportingOwner/reportingOwnerRelationship/isOfficer').text == '1':
                isOfficer = True
            elif tree.find('reportingOwner/reportingOwnerRelationship/isOfficer').text == '0':
                isOfficer = None
            elif tree.find('reportingOwner/reportingOwnerRelationship/isOfficer').text == 'false':
                isOfficer = None
            else:
                isOfficer = tree.find('reportingOwner/reportingOwnerRelationship/isOfficer')
                isOfficer = isOfficer.text               
                
            if tree.find('reportingOwner/reportingOwnerRelationship/isTenPercentOwner')== None:
                isTenPercentOwner = None
            elif tree.find('reportingOwner/reportingOwnerRelationship/isTenPercentOwner').text == '1':
                isTenPercentOwner = True
            elif tree.find('reportingOwner/reportingOwnerRelationship/isTenPercentOwner').text == '0':
                isTenPercentOwner = None
            elif tree.find('reportingOwner/reportingOwnerRelationship/isTenPercentOwner').text == 'false':
                isTenPercentOwner = None
            else:
                isTenPercentOwner = tree.find('reportingOwner/reportingOwnerRelationship/isTenPercentOwner')
                isTenPercentOwner = isTenPercentOwner.text                
                
            if tree.find('reportingOwner/reportingOwnerRelationship/officerTitle')== None:
                officerTitle = None
            else:
                officerTitle = tree.find('reportingOwner/reportingOwnerRelationship/officerTitle').text

                
            if tree.find('reportingOwner/reportingOwnerId/rptOwnerName')== None:
                rptOwnerName = None
            else:
                rptOwnerName = tree.find('reportingOwner/reportingOwnerId/rptOwnerName').text
 
            if tree.find('ownerSignature/signatureName')== None:
                signatureName = None
            else:
                signatureName = tree.find('ownerSignature/signatureName').text
                signatureNameLower=str.lower(signatureName)
                if signatureNameLower[:3]== "/s/":
                    signatureName=signatureName[3:] # remove the /s/
                    signatureName=signatureName.strip() # remove the white spaces
            
            if tree.find('periodOfReport')==None:
                periodOfReport = None
            else:
                periodOfReport= tree.find('periodOfReport').text

            
            if signatureNameLower.find('attorney')>0: # was it filed by an Attorney?
                Attorney=True
            else:
                Attorney=False
 
            #Direct own or Inderect own (like a trust or wife)
            DorI = tree.findall('nonDerivativeTable/nonDerivativeTransaction/ownershipNature/directOrIndirectOwnership/value')
            if DorI == None:
                DorI = []
                
            transactionCode = tree.findall('nonDerivativeTable/nonDerivativeTransaction/transactionCoding/transactionCode')
            if transactionCode == None:
                transactionCode = []
            tradingSymbol = tree.find('issuer/issuerTradingSymbol').text
            tradingSymbolUpper=str.upper(str(tradingSymbol))
            tradingName = tree.find('issuer/issuerName').text
            transactionShares = tree.findall('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionShares/value')
            if transactionShares == None:
                transactionShares = []
            transactionPricePerShare = tree.findall('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionPricePerShare/value')
            if transactionPricePerShare == None:
                transactionPricePerShare = []
            if transactionShares == None:
                transactionShares = []
            
            postTransactionAmounts = tree.findall('nonDerivativeTable/nonDerivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')    
            if transactionShares == None:
                transactionShares = []           
            
            

            for price, shares, direct, code, PostShares in zip(transactionPricePerShare, transactionShares, DorI, transactionCode,postTransactionAmounts):
                if direct.text == 'D' and code.text == 'P':
                    TotalValue = TotalValue + float(shares.text)*float(price.text)
                    DorIList.append(direct.text)
                    tranPPSlist.append(float(price.text))
                    tranSlist.append(float(shares.text))
                    transactionCodeList.append(code.text)
                    BeforeSharesList.append(float(PostShares.text)-float(shares.text))
                    PostSharesList.append(float(PostShares.text))
                elif direct.text == 'D' and code.text == 'S':
                    TotalValue = TotalValue - float(shares.text)*float(price.text)
                    DorIList.append(direct.text)
                    tranPPSlist.append(float(price.text))
                    tranSlist.append(float(shares.text)*-1)
                    transactionCodeList.append(code.text)
                    BeforeSharesList.append(float(PostShares.text)+float(shares.text))
                    PostSharesList.append(float(PostShares.text))                   
                else:
                    pass
            
            
#             for code in transactionCode:
#                 transactionCodeList.append(code.text)
#             for item in DorI:
#                 DorIList.append(item.text)
            

            Entry = [link,today,periodOfReport,tradingName,tradingSymbolUpper,TotalValue, isDirector,isOfficer,isOfficer,officerTitle,rptOwnerName,signatureName,Attorney]
            if TotalValue != 0:
            #If the transation code was either P or S Total value will not be 0
            #If so get the current stock value
                #print (Entry)
                
                if TotalValue > 0:
                    # if they are purchasing.... shares how many did they have before
                    BeforeShares= min(BeforeSharesList)
                    AcquiredOrDisposed="A" # Net buying stock
                else: # if they are selling...
                    BeforeShares= max(BeforeSharesList)
                    AcquiredOrDisposed="D" # Selling stock
                    
                    
                Entry.append(AcquiredOrDisposed)    
                Entry.append(BeforeShares)
                Entry.append(PostSharesList[-1])# How many shares they own once it is all said and done
                
                totshares=avgPPS=0
                totshares=sum(tranSlist) # Calculate the total number of shares
                avgPPS=sum(tranPPSlist)/len(tranPPSlist) # Average price for those shares
                #print(totshares,avgPPS)
                Entry.append(totshares)
                Entry.append(avgPPS)
                
                
                #---------------------------
                # Now lets go get the last/current Trading Value
                ticker = tradingSymbol.lower()
                res = requests.get('http://www.nasdaq.com/symbol/' + ticker)
                soup = bs4.BeautifulSoup(res.text, 'html.parser')
                span = str(soup.find_all("div", attrs={"id":"qwidget_lastsale"})) # Find where the that has the ID = LasteSale

                #print (span)
                if len(span) <3:# filtering out if it is empty
                    current_price= None
                else:
                    current_price=float(span[span.find('>')+2:span.find('<',span.find('>'))])
                    #Get that value of that span and save it as a float, starting at he first > and ending at the next <
                #print ("Current Price " + str(current_price))
                Entry.append(current_price)
                
                Feedresults=GoogleNewsFeed(tradingSymbolUpper) # go see what google news thinks
                Entry.append(Feedresults[0]) # Average Polarity
                Entry.append(Feedresults[1]) # Average Subjectifity
                print (Entry)
                #Appned this Entery to the file
                #fd = open('Enteries.csv','a')
                with open('Enteries.csv', 'ab') as f:
                    writer = csv.writer(f)
                    writer.writerow(Entry)
                    #svwriter.writerow(Entry)
                #fd.write(str(Entry))
                #fd.close()


def edgar_feed(url):
    try:
        d = feedparser.parse(url)
        time.sleep(1)

        for entry in range(0,99): #Filter the Company name
          
            company_name = d.entries[entry].title.lower()
            company_name = company_name.split('- ')
            company_name = company_name[1].split(' (')
            company_name = company_name[0]
            company_name = company_name.replace('.', '')
            company_name = company_name.replace(',', '')
            if '&amp;' in company_name:
                company_name = company_name.replace('&amp;', '&')
                
#            print( "Company: " + company_name)

            if d.entries[entry].title[0:1:] == '4': # Only get From 4's 
#           if company_name in lower and d.entries[entry].title[0:1:] == '4':
                link = d.entries[entry].link # slects the link the individual filing
                last50 = slice(-50, None) 
#                print( "Company: " + company_name)
#                print (link)
#                print (link[last50])
                if link[last50] not in stocks_sent: # Keeps you from getting the same most resent one again and again.
#                    print ('found something new!')
                    scrape_xml(link)
                    stocks_sent.append(link[last50])
                    
            else:
                pass
    except Exception as e: # Report an error
        print (e) 
        print (datetime.datetime.today())
        pass
#-------------------------------------------------------------------------------------#
#SCRIPT BODY
#Stock screen: Market Cap < $500m, D/E < 1.5, P/E < 1.5
    
url = 'http://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&type=&company=&dateb=&owner=only&start=0&count=100&output=atom'
print ('monitoring feed...')
run_counter = 0
def job():
    global run_counter
    time.sleep(10)
    run_counter += 1
    #print ("Run Counter: " + str(run_counter))
    if run_counter % 100 == 0:
        print ('Completed ' + str(run_counter) + ' passes.')
        print ('--------------')
    edgar_feed(url)


while True:
    job()

monitoring feed...
a bytes-like object is required, not 'str'
2017-03-17 20:01:08.434655


KeyboardInterrupt: 